In [26]:
#! /usr/bin/python
import re
import numpy
import numbers

In [78]:
def load_gate( gate ):
    instruction, wire = gate[0]
    if not any(x in instruction for x in ['OR', 'AND', 'LSHIFT', 'RSHIFT', 'NOT']):
        instruct = [wire, [" ", "WIRE", instruction.split(' ')[0]]]
        try:
            instruct[1][2] = numpy.uint16(int(instruct[1][2]))
        except ValueError:
            print("create_gate: oops")
        if isinstance( instruct[1][2], numpy.uint16 ):
            instruct[1][1] = "SOURCE"
        print (instruct)
        gates.append(instruct)
    if any(x in instruction for x in ['OR', 'AND']):
        instruct = [wire, instruction.split(' ')]
        gates.append(instruct)
    if any(x in instruction for x in ['LSHIFT', 'RSHIFT']):
        instruct = [wire, instruction.split(' ')]
        gates.append(instruct)
    if any(x in instruction for x in ['NOT']):
        instruct = [wire, [" ", instruction.split(' ')[0], instruction.split(' ')[1]]]
        gates.append(instruct)

In [79]:
def create_wire( wire, value ):
    wires[wire] = numpy.uint16(value)
    print ("CREATING WIRE:  ", wire, " with value :", value)

In [80]:
def execute_gate ( gate ):
    if (gate[1][1]) == 'AND':
        create_wire(gate[0], numpy.uint16(wires[gate[1][0]]) & numpy.uint16(wires[gate[1][2]]))
    if (gate[1][1]) == 'OR':
        create_wire(gate[0], numpy.uint16(wires[gate[1][0]]) | numpy.uint16(wires[gate[1][2]]))
    if (gate[1][1]) == 'LSHIFT':
        create_wire(gate[0], numpy.uint16(wires[gate[1][0]]) << int(gate[1][2]))
    if (gate[1][1]) == 'RSHIFT':
        create_wire(gate[0], numpy.uint16(wires[gate[1][0]]) >> int(gate[1][2]))
    if (gate[1][1]) == 'NOT':
        create_wire(gate[0], ~numpy.uint16(wires[gate[1][2]]))
    if (gate[1][1]) == 'WIRE':
        create_wire(gate[0], numpy.uint16(wires[gate[1][2]]))

In [81]:
def find_candidate( gates ):
    for gate in gates:
        if ((gate[1][1]) == 'AND'):
            if (gate[1][0] in wires) and (gate[1][2] in wires):
                exec_gate = gate
                gates.remove(exec_gate)
                break
        if (gate[1][1]) == 'OR':
            if (gate[1][0] in wires) and (gate[1][2] in wires):
                exec_gate = gate
                gates.remove(exec_gate)
                break
        if (gate[1][1]) == 'LSHIFT':
            if (gate[1][0] in wires):
                exec_gate = gate
                gates.remove(exec_gate)
                break            
        if (gate[1][1]) == 'RSHIFT':
            if (gate[1][0] in wires):
                exec_gate = gate
                gates.remove(exec_gate)
                break            
        if (gate[1][1]) == 'NOT':
            if (gate[1][2] in wires):
                exec_gate = gate
                gates.remove(exec_gate)
                break
        if (gate[1][1]) == 'WIRE':
            if (gate[1][2] in wires):
                exec_gate = gate
                gates.remove(exec_gate)
                break
    return exec_gate


In [82]:
match_gate = re.compile('(^[a-zA-Z0-9 ]*) -> ([a-z]*)')

#filename = './input' #first star
filename = './input2' #second star
wires = {}
gates = []

print ("test")
#    READ ALL GATES
with open(filename) as f:
    for linen in f:
        line = linen.rstrip('\n')
        if not (match_gate.findall(line) == []):
            load_gate (match_gate.findall(line))
    
#    SETUP ALL SOURCES
create_wire('1', 1)
delgates = []
for gate in gates:
    if gate[1][1] == "SOURCE":
        create_wire(gate[0][0], gate[1][2])
        print("setup", gate)
        delgates.append(gate)

for gate in delgates:
        gates.remove(gate)

print ("-----------")


#    FIND GATES WITH COMPLETE SOURCES AND EXECUTE
while not (gates == []):
    print ("---\n# GATES LEFT:   ", len(gates))
    candidate = find_candidate( gates )
    print ("EXECUTING GATE: ", candidate[1][0], candidate[1][1], candidate[1][2], " -> ", candidate[0])
    execute_gate ( candidate )
     

test
['b', [' ', 'SOURCE', 16076]]
create_gate: oops
['a', [' ', 'WIRE', 'lx']]
['c', [' ', 'SOURCE', 0]]
CREATING WIRE:   1  with value : 1
CREATING WIRE:   b  with value : 16076
setup ['b', [' ', 'SOURCE', 16076]]
CREATING WIRE:   c  with value : 0
setup ['c', [' ', 'SOURCE', 0]]
-----------
---
# GATES LEFT:    337
EXECUTING GATE:  c LSHIFT 1  ->  t
CREATING WIRE:   t  with value : 0
---
# GATES LEFT:    336
EXECUTING GATE:  b RSHIFT 1  ->  v
CREATING WIRE:   v  with value : 8038
---
# GATES LEFT:    335
EXECUTING GATE:  b RSHIFT 5  ->  f
CREATING WIRE:   f  with value : 502
---
# GATES LEFT:    334
EXECUTING GATE:  b RSHIFT 2  ->  d
CREATING WIRE:   d  with value : 4019
---
# GATES LEFT:    333
EXECUTING GATE:  b RSHIFT 3  ->  e
CREATING WIRE:   e  with value : 2009
---
# GATES LEFT:    332
EXECUTING GATE:  e OR f  ->  g
CREATING WIRE:   g  with value : 2047
---
# GATES LEFT:    331
EXECUTING GATE:  e AND f  ->  h
CREATING WIRE:   h  with value : 464
---
# GATES LEFT:    330
EXECUT

In [83]:
print ("a: ", wires['a'])

a:  2797
